In [1]:
# make imports

In [2]:
# import datasets (disasters)

# Q1: Given a year is between 2000-2018, what is the chance that it's number of total disasters is greater than the average number of disasters for all years 1900-2018?

# Q2: What is the probability distribution of natural disasters over the years 1900-1999? 2000-2018?

# Q3: What % of natural disasters happen before 1925? 1950? 1975? 2000? 2018?

# Q4: By the decade: How does the mean number of annual natural disasters change?

# Q5: By the decade:  How does the median number of annual natural disasters change?¶

# Q6: At what number of natural disasters, did 75% of years 1900-2000 have less than that? Of years 2000-2018?

# Q7: At what economic cost, did 75% of years 1900-2000 experience than that (due to natural disasters)? Of years 2000-2018?

# Q8 Null Hypothesis: A greater average number of CO2 emissions from 2000-2018 is responsible from greater mean number of natural disasters annually from 2000-2018.

# Do we accept or reject this?